In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import torch
import pickle
import random
import math
from tqdm import tqdm
from torch import nn, optim
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, util

import load_data
from load_data import GenderDataset, gender_data_collate_fn
from models.encoder_t5 import EncoderT5
from models.classifier_bert import ClassifierBERT
from models.perplexity_gpt import PerplexityGPT2
from models.similarity_sent_enc import encode_for_similarities

In [3]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:2')

print(device)

cuda:2


In [4]:
# Hyper parameters
num_epoch = 100
batch_size = 32
lr = 1e-4
wd = 5e-6
print_every = 500

In [5]:
with open(os.path.join(os.curdir, "data", "blog.json"), "r") as file:
    json_data = json.load(file)
docs = json_data['docs'][1:] # I don't want to see the first document

In [6]:
num_docs = len(docs)
num_train_docs = int(num_docs * 0.7)
num_val_docs = int(num_docs * 0.15)
num_test_docs = num_docs - num_train_docs - num_val_docs
print(num_train_docs, num_val_docs, num_test_docs)

13773 2951 2952


In [7]:
train_docs = docs[:num_train_docs]
val_docs = docs[num_train_docs:num_train_docs+num_val_docs]
test_docs = docs[num_train_docs+num_val_docs:]

In [8]:
train_dataset, val_dataset, test_dataset = None, None, None
load_from_pickled = False

if os.path.exists(os.path.join(os.curdir, "data", "train.pickle")):
    load_from_pickled = True
    with open(os.path.join(os.curdir, "data", "train.pickle"), "rb") as f:
        train_dataset = pickle.load(f)
    with open(os.path.join(os.curdir, "data", "val.pickle"), "rb") as f:
        val_dataset = pickle.load(f)
    with open(os.path.join(os.curdir, "data", "test.pickle"), "rb") as f:
        test_dataset = pickle.load(f)
else:
    train_dataset = GenderDataset(train_docs)
    val_dataset = GenderDataset(val_docs)
    test_dataset = GenderDataset(test_docs)

if not load_from_pickled:
    with open(os.path.join(os.curdir, "data", "train.pickle"), "wb") as f:
        pickle.dump(train_dataset, f)
    with open(os.path.join(os.curdir, "data", "val.pickle"), "wb") as f:
        pickle.dump(val_dataset, f)
    with open(os.path.join(os.curdir, "data", "test.pickle"), "wb") as f:
        pickle.dump(test_dataset, f)

print(load_from_pickled)   

True


In [9]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8,
    collate_fn=gender_data_collate_fn
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8,
    collate_fn=gender_data_collate_fn
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8,
    collate_fn=gender_data_collate_fn
)

In [10]:
next(enumerate(test_dataloader))

(0,
 (tensor([[ 3104,  2065,  1049,  ...,  2729,  2013,  1016],
          [ 2030,  7137,  2007,  ...,  1016,  1032,  1032],
          [17437,  2001,  3263,  ...,  6191,  1054,  1009],
          ...,
          [ 2232,  2013,  1009,  ...,  2065,  2176,  1016],
          [ 2041,  3963,  1014,  ...,  4071,  2021,  1016],
          [ 1014,  1049,  2083,  ...,  2251,  1016, 17407]], dtype=torch.int32),
  tensor([128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,
          128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,
          128, 128, 128, 128], dtype=torch.int32),
  tensor([0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
          0, 0, 1, 0, 1, 0, 1, 1])))

In [11]:
encoder_model = EncoderT5(
    vocab_size=load_data.tokenizer.vocab_size
).to(device)

classifier_model = ClassifierBERT(
    vocab_size=load_data.tokenizer.vocab_size
).to(device)

ppl_model = PerplexityGPT2(
    vocab_size=load_data.tokenizer.vocab_size,
    bos_token_id=load_data.tokenizer.bos_token_id,
    eos_token_id=load_data.tokenizer.eos_token_id,
).to(device)

st_model = SentenceTransformer('all-mpnet-base-v2')
st_model[0].auto_model=st_model[0].auto_model.to(device)

In [18]:
classifier_model.load_state_dict(torch.load('./save/cls_model_271_128_epoch_0.file'))
ppl_model.load_state_dict(torch.load('./save/ppl_model_22_48_epoch_0.file'))
encoder_model.load_state_dict(torch.load('./save/enc_model_280_32_epoch_0.file'))

<All keys matched successfully>

In [13]:
optimizer = optim.AdamW(encoder_model.parameters(), lr=lr, weight_decay=wd)

In [14]:
criterion_similarity = nn.MSELoss()
criterion_classification = nn.CrossEntropyLoss()
criterion_perplexity = nn.CrossEntropyLoss() # Placeholder

In [15]:
def train(train_dataloader, val_dataloader, models, criterions, optimizer, num_epoch):
    
    train_id = random.randint(0, 1000)
    log = open(f'./save/enc_{train_id}.txt', 'w')
    models['st'].eval()
    models['ppl'].eval()
    models['cls'].eval()

    for epoch in range(num_epoch):
        
        print(f"Epoch {epoch}, total {len(train_dataloader)} batches\n")
        log.write(f"Epoch {epoch}, total {len(train_dataloader)} batches\n")
        log.flush()

        models['enc'].train()

        for batch, (src_ids, src_len, tgt) in enumerate(train_dataloader):

            torch.cuda.empty_cache()
            optimizer.zero_grad()
            src_ids = src_ids.to(device)
            tgt = torch.full((batch_size, 2), 0.5).to(device)

            # Calculate the obfuscated logits via the encoder & softmax
            obf_logits = models['enc'](src_ids)
            obf_logits = nn.functional.softmax(obf_logits, dim=-1)
            
            # Calculate similarity loss
            result = encode_for_similarities(models['st'], device, src_ids, obf_logits)
            loss_sim = torch.mean(result)

            # Calculate gender classification loss aiming for 50/50
            gender_logits = models['cls'](obf_logits)
            loss_cls = criterions['cls'](gender_logits, tgt)
            
            # Calculate perplexity loss
            ppl_logits = models['ppl'](obf_logits)
            shift_ppl = ppl_logits[..., :-1, :].contiguous()
            shift_obf = obf_logits[..., 1:, :].contiguous()
            loss_ppl = criterions['ppl'](shift_ppl.view(-1, shift_ppl.size(-1)), shift_obf.view(-1, shift_obf.size(-1)))

            loss = loss_sim * 1 + loss_cls * 0 + loss_ppl * 1
            loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()
            
            if batch % print_every == 0:
                print(f"Epoch Step: {batch} Sim Loss: {loss_sim} Class Loss: {loss_cls} PPL Loss: {loss_ppl}")
                log.write(f"Epoch Step: {batch} Sim Loss: {loss_sim} Class Loss: {loss_cls} PPL Loss: {loss_ppl}\n")

                torch.save(models['enc'].state_dict(), f'./save/enc_model_{train_id}_{batch_size}_epoch_{epoch}.file')

            if batch % (print_every * 5) == 0:
                print(f"\nBegin Evaluation")
                models['enc'].eval()
                total_acc = 0
                total_ppl = 0
                total_sim = 0
                limit=len(val_dataloader)
                if limit>100: 
                    limit=100
                with torch.no_grad():
                    for batch, (src_ids, src_len, tgt) in tqdm(enumerate(val_dataloader),total=limit):
                        src_ids = src_ids.to(device)
                        tgt = tgt.to(device)

                        obf_logits = models['enc'](src_ids)
                        obf_logits = nn.functional.softmax(obf_logits, dim=-1)
                        
                        result = encode_for_similarities(models['st'], device, src_ids, obf_logits)
                        gender_logits = models['cls'](obf_logits)

                        ppl_logits = models['ppl'](obf_logits)
                        shift_ppl = ppl_logits[..., :-1, :].contiguous()
                        shift_obf = obf_logits[..., 1:, :].contiguous()
                        loss_ppl = criterions['ppl'](shift_ppl.view(-1, shift_ppl.size(-1)), shift_obf.view(-1, shift_obf.size(-1)))

                        total_acc += (gender_logits.argmax(1) == tgt).sum().item()
                        total_ppl += loss_ppl
                        total_sim += torch.mean(result)

                        if batch >= limit:
                            break

                acc = total_acc / limit / batch_size
                ppl = math.exp(total_ppl / limit)
                sim = total_sim / limit

                print(f"Validation Accuracy: {acc}\nValidation Perplexity: {ppl}\nValidation Semantic Similarity: {sim}\n")
                log.write(f"Validation Accuracy: {acc}\nValidation Perplexity: {ppl}\nValidation Semantic Similarity: {sim}\n")
                log.flush()


In [16]:
train(
    train_dataloader,
    val_dataloader,
    {
        'enc': encoder_model,
        'cls': classifier_model,
        'st': st_model,
        'ppl': ppl_model,
    },
    {
        'sim': criterion_similarity,
        'cls': criterion_classification,
        'ppl': criterion_perplexity,
    },
    optimizer,
    num_epoch
)

Epoch 0, total 14042 batches

Epoch Step: 0 Sim Loss: 0.15747246146202087 Class Loss: 1.07301926612854 PPL Loss: 13.986932754516602

Begin Evaluation


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:27<00:00,  3.62it/s]


Validation Accuracy: 0.4534375
Validation Perplexity: 1294072.9525510643
Validation Semantic Similarity: 0.14865118265151978

Epoch Step: 500 Sim Loss: 0.12089000642299652 Class Loss: 1.436525821685791 PPL Loss: 0.8189070820808411
Epoch Step: 1000 Sim Loss: 0.14016860723495483 Class Loss: 1.4365272521972656 PPL Loss: 0.8162006735801697
Epoch Step: 1500 Sim Loss: 0.15518787503242493 Class Loss: 1.4365277290344238 PPL Loss: 0.8154993057250977
Epoch Step: 2000 Sim Loss: 0.1530798375606537 Class Loss: 1.4365277290344238 PPL Loss: 0.8152191638946533
Epoch Step: 2500 Sim Loss: 0.16115376353263855 Class Loss: 1.436527967453003 PPL Loss: 0.8150780200958252

Begin Evaluation


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:27<00:00,  3.59it/s]


Validation Accuracy: 0.5478125
Validation Perplexity: 2.2778441151444695
Validation Semantic Similarity: 0.14979593455791473

Epoch Step: 3000 Sim Loss: 0.15263764560222626 Class Loss: 1.436527967453003 PPL Loss: 0.8149926066398621
Epoch Step: 3500 Sim Loss: 0.1657954752445221 Class Loss: 1.436527967453003 PPL Loss: 0.8149411678314209
Epoch Step: 4000 Sim Loss: 0.15140114724636078 Class Loss: 1.436527967453003 PPL Loss: 0.814909815788269
Epoch Step: 4500 Sim Loss: 0.15152421593666077 Class Loss: 1.436527967453003 PPL Loss: 0.814879834651947
Epoch Step: 5000 Sim Loss: 0.1381053328514099 Class Loss: 1.436527967453003 PPL Loss: 0.8148666024208069

Begin Evaluation


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:27<00:00,  3.61it/s]


Validation Accuracy: 0.5553125
Validation Perplexity: 2.277357567115073
Validation Semantic Similarity: 0.14946451783180237

Epoch Step: 5500 Sim Loss: 0.13133494555950165 Class Loss: 1.436527967453003 PPL Loss: 0.8148579001426697
Epoch Step: 6000 Sim Loss: 0.12926816940307617 Class Loss: 1.436527967453003 PPL Loss: 0.8148557543754578


KeyboardInterrupt: 

In [16]:
def test(test_dataloader, models, criterions):
    
    models['st'].eval()
    models['ppl'].eval()
    models['cls'].eval()
    models['enc'].eval()
    
    print(f"\nBegin Testing")
    total_acc = 0
    total_ppl = 0
    total_sim = 0
    limit=len(test_dataloader)
    if limit>100: 
        limit=100
    with torch.no_grad():
        for batch, (src_ids, src_len, tgt) in tqdm(enumerate(test_dataloader),total=limit):
            src_ids = src_ids.to(device)
            tgt = tgt.to(device)

            obf_logits = models['enc'](src_ids)
            obf_logits = nn.functional.softmax(obf_logits, dim=-1)

            result = encode_for_similarities(models['st'], device, src_ids, obf_logits)
            gender_logits = models['cls'](obf_logits)

            ppl_logits = models['ppl'](obf_logits)
            shift_ppl = ppl_logits[..., :-1, :].contiguous()
            shift_obf = obf_logits[..., 1:, :].contiguous()
            loss_ppl = criterions['ppl'](shift_ppl.view(-1, shift_ppl.size(-1)), shift_obf.view(-1, shift_obf.size(-1)))

            print(test_dataset.idx2str(src_ids[0].cpu().detach()))
            print(test_dataset.idx2str(obf_logits[0].argmax(-1).cpu().detach()))

            total_acc += (gender_logits.argmax(1) == tgt).sum().item()
            total_ppl += loss_ppl
            total_sim += torch.mean(result)

            if batch >= limit:
                break

    acc = total_acc / limit / batch_size
    ppl = math.exp(total_ppl / limit)
    sim = total_sim / limit

    print(f"Test Accuracy: {acc}\nTest Perplexity: {ppl}\nTest Semantic Similarity: {sim}\n")

In [19]:
test(
    test_dataloader,
    {
        'enc': encoder_model,
        'cls': classifier_model,
        'st': st_model,
        'ppl': ppl_model,
    },
    {
        'sim': criterion_similarity,
        'cls': criterion_classification,
        'ppl': criterion_perplexity,
    }
)


Begin Testing


  1%|▊                                                                                  | 1/100 [00:00<00:37,  2.62it/s]

in me and beth. mostly beth. - rrb - made up the feature! the feature! its crazy! i mean we normally wouldnt be doing that but we did and its better than the crap she made up! mrs. winn came and we showed her our routine and she loved it but ginny sat there all smug. i was like okay whatever. and she was trying to get us to make up stuff for the next show, which might i remind you we have 3 weeks to work on, when she didnt have anything for 2 of the other songs! i was like well why dont we make up those first
[UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK]

  2%|█▋                                                                                 | 2/100 [00:00<00:31,  3.14it/s]

according to this [UNK] article, a swiss court has decided that a group of gypsies who lost family members during the holocaust can rightfully sue ibm for providing punch - card tabulating machines that [UNK] the extermination campaigns of the nazis. this is a particularly interesting connection, in my opinion, since the lawsuit is coming from one of the most underrepresented groups in holocaust reparations - a group that is often denigrated for its lack of education and clan - type family [UNK]s. yet it is a [UNK] group who is suing ibm on very technical grounds. i'll be curious to see how this
[UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [

...

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:27<00:00,  3.65it/s]

, because it seems like i should have died in this one, and i'm a big [UNK] of life going full - circle. i suppose i don't really have a say in the matter, but if you were going to make a movie about my life, kill me off in a car accident at the end. it will make me happy. i ended up a happy baby after that, too. i started doing the things i love now back then, as you can see from the above picture. these include wearing wife - beaters and laying around on the floor in my underwear, as well as being mildly overweight and
[UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK]

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:27<00:00,  3.60it/s]

- lrb - an old timer at the bar stated " the disco's good - it's not too loud and there are tunes you can really hum to! " - rrb -. the bride and groom left for mexico, helen [UNK] couldn't stop speaking french and i can't remember much else. sorry it's a bit long winded and probably a load of nonsense but i'm not gifted like paul carr, shakespeare or jilly cooper. as a final note, i want to thank rich and el for a fantastic and enjoyable day. i was humbled to have been invited and i wish
[UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [U

Test Accuracy: 0.485
Test Perplexity: 2.9595785824245464
Test Semantic Similarity: 0.14985410869121552

